In [1]:
from IPython.display import clear_output

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

import pandas as pd
import numpy as np

import json
import time
import mouse
import os
from os.path import exists


### Парсим последний уровень категоризатора по ссылкам, собранным на предыдущем этапе

In [3]:
def bring_me_YSCaptcha():
    '''
    Вызывает вылет капчи
    '''
    while not driver.find_elements(by='id', value='js-button'):
        url = "https://market.yandex.ru/catalog--igrushki-dlia-detei-do-3-let/59693"
        driver.get(url)

        
def handle_if_YSCaptcha(url:str,
                        x1:int=1021, 
                        x2:int=1284, 
                        y1:int=534, 
                        y2:int=575):
    '''
    Автоматизация для обхода капчи (Параметры функции -- координаты 
    прямоугольника, ограничивающего рабочую область кнопки для прохождения капчи)
    Использовать mouse.get_position() для подбора координат
    '''
    counter = 0
    while driver.find_elements(by='id', value='js-button'):
        print('Работаем с капчей')
        if counter:
            driver.get(url)        
        x = np.random.randint(x1, x2)
        y = np.random.randint(y1, y2)
        mouse.move(x, y, absolute=True, duration=0.1)
        mouse.click('left')
        time.sleep(2.5)
        counter += 1
        

def check_if_loaded(url:str='', lag_range:int=3):
    '''
    Декоратор для проверки страницы на ошибки при загрузке
    '''
    def decorator(func):
        def wrapper(*args, **kwargs):
            counter = 0
            for reload in range(2):
                if not counter:
                    pass
                elif not counter % lag_range:
                    print(url)
                    driver.get(url)
                    handle_if_YSCaptcha(url)
                else:
                    break

                for lag in range(lag_range):
                    counter += 1
                    time.sleep(lag)
                    tree_lvl3, lvl3_href = func(*args, **kwargs)
                    if tree_lvl3:
                        break

            return tree_lvl3, lvl3_href
        return wrapper
    return decorator

@check_if_loaded(url='a')
def parse_item_names():
    '''
    Функция для сбора наименований и цены со страницы
    '''
    names_xpath = '//*[@data-autotest-id="product-snippet"]/div[1]|div[2]/h3/a'
#         values_xpath = '//*[@data-autotest-id="product-snippet"][1]//*[@data-auto="price-value"] | //*[@data-autotest-id="product-snippet"]/div[2]/div[1]/div/div/div[1]/a/div/div/div[2]/div[1]/div/h3 | //*[@data-autotest-id="product-snippet"][1]/div/div/a/div/span/span[1]'

    names_xpath = '//*[@data-autotest-id="product-snippet"]/div[2]/h3/a'
#         values_xpath = '//*[@data-autotest-id="product-snippet"]/div/div/div/div[1]/div/h3 | //*[@data-autotest-id="product-snippet"]/div/div/a/div'

    names_xpath = '//*[@data-autotest-id="product-snippet"]/div[1]/h3/a'
#         values_xpath = '//*[@data-autotest-id="product-snippet"]/div/div/div/div/div/h3'


    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    button = driver.find_elements(by='xpath', value="//*[text()='Показать ещё']")
    if button:
        button[0].click()
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(0.7)

    time.sleep(1)
    while True:
        try:
            names = driver.find_elements(by='xpath', value=names_xpath)
            names = [name.text for name in names]
#                 values = driver.find_elements(by='xpath', value=values_xpath)
#                 values = [value.text for value in values]
            values = [0]*len(names)
            if len(names) == len(values):
                break
        except StaleElementReferenceException:
            pass

        clear_output()
        print('Элементы не прогрузились(')
        print('Пробуем снова...')
        time.sleep(0.5)

    return names, values


def special_button(text:str='Да, мне есть 18 лет'):
    '''
    Автоматических обход подтверждения возраста 
    '''
    special_buttons = driver.find_elements('xpath', f"//*[text()='{text}']")
    if special_buttons:
        special_buttons[0].find_element('xpath', '..').click()

## Подгрузка полного справочника категорий

In [4]:
# Подгрузка ручноый валидации
our_cats = pd.read_excel('./data/primal_labeling_validation.xlsx')
selected_names = our_cats['near_product'].str.replace(r'\[.*\] ', '', regex=True).tolist()

In [6]:
# Подгрузка полного категоризатора
yandex_cats = pd.read_excel('./data/full_YM_categories.xlsx')

yandex_cats['level_3'] = yandex_cats.apply(lambda x: x['level_3'] if x['level_3'] is not np.nan else x['level_2'], axis = 1)
yandex_cats['lvl3_href'] = yandex_cats.apply(lambda x: x['lvl3_href'] if x['lvl3_href'] is not np.nan else x['lvl2_href'], axis = 1)


In [7]:
selected_values = yandex_cats[yandex_cats['level_3'].isin(selected_names)][['level_3', 'lvl3_href']].drop_duplicates()
selected_values.groupby('level_3', as_index=False).count().query('lvl3_href > 3').sum()
selected_values

,level_3,lvl3_href
0,Смартфоны,https://market.yandex.ru/catalog--smartfony/61808
2,Наушники и Bluetooth-гарнитуры,https://market.yandex.ru/catalog--naushniki-i-...
3,Портативная акустика,https://market.yandex.ru/catalog--portativnaia...
5,Телевизоры,https://market.yandex.ru/catalog--televizory/6...
6,ТВ-приставки и медиаплееры,https://market.yandex.ru/catalog--tv-pristavki...
...,...,...
6998,Раскраски,https://market.yandex.ru/catalog--raskraski/50...
7000,Настольные игры,https://market.yandex.ru/catalog--nastolnye-ig...
7003,Аминокислоты,https://market.yandex.ru//catalog--aminokislot...
7008,Женское здоровье,https://market.yandex.ru//catalog--zhenskoe-zd...


In [15]:
# Список страниц для парсинга
url_list = selected_values['lvl3_href'].tolist()

## Парсим item_name'ы

In [9]:
# Подключаем драйвер
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)

In [11]:
if os.path.exists('counter_item_names.txt'):
    with open('counter_item_names.txt') as f:
        i = int(f.readline())
else:
    i = 0
i

0

In [ ]:
# Парсинг цен не доработан из-за варьируемости положения скидок и вида карточек товаров
file_name = './data/YM_item_sample.csv'

# Парсер
for url in url_list[i:]:
#     Загружаем страницу, проверяем капчу
    driver.get(url)
    handle_if_YSCaptcha(url)
    special_button()
    special_button('Сбросить фильтры')
    time.sleep(2)
    special_buttons = driver.find_elements('xpath', f"//*[text()='Сбросить фильтры']")
    if special_buttons:
        pd.DataFrame(data = {'lvl3_href': [url],
                     'item_name': 'нет данных',
                     'value': 'нет данных'}).to_csv(file_name, 
                                                         mode='a' if exists(file_name) else 'w', 
                                                         sep='\t', 
                                                         index=False,
                                                         header = not exists(file_name))
        i += 1
        with open('counter.txt', 'w') as f:
            f.write(str(i))
        time.sleep(1)
        continue

    
    print(f'Парсим {i} элемент...')
    
    @check_if_loaded(url)
    def parse_item_names():
        '''
        Функция для сбора наименований и цены со страницы
        '''
        names_xpath = '//*[@data-autotest-id="product-snippet"]/div[1]|div[2]/h3/a'
#         values_xpath = '//*[@data-autotest-id="product-snippet"][1]//*[@data-auto="price-value"] | //*[@data-autotest-id="product-snippet"]/div[2]/div[1]/div/div/div[1]/a/div/div/div[2]/div[1]/div/h3 | //*[@data-autotest-id="product-snippet"][1]/div/div/a/div/span/span[1]'

        names_xpath = '//*[@data-autotest-id="product-snippet"]/div[2]/h3/a'
#         values_xpath = '//*[@data-autotest-id="product-snippet"]/div/div/div/div[1]/div/h3 | //*[@data-autotest-id="product-snippet"]/div/div/a/div'

        names_xpath = '//*[@data-autotest-id="product-snippet"]/div[1]/h3/a'
#         values_xpath = '//*[@data-autotest-id="product-snippet"]/div/div/div/div/div/h3'

        
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        button = driver.find_elements(by='xpath', value="//*[text()='Показать ещё']")
        if button:
            button[0].click()
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(0.7)

        time.sleep(1)
        while True:
            try:
                names = driver.find_elements(by='xpath', value=names_xpath)
                names = [name.text for name in names]
#                 values = driver.find_elements(by='xpath', value=values_xpath)
#                 values = [value.text for value in values]
                values = [0]*len(names)
                if len(names) == len(values):
                    break
            except StaleElementReferenceException:
                pass

            clear_output()
            print('Элементы не прогрузились(')
            print('Пробуем снова...')
            time.sleep(0.5)

        return names, values
        
#     Парсим данные со страницы
    names, values = parse_item_names()
    
    if not names:
        pd.DataFrame(data = {'lvl3_href': [url],
                     'item_name': 'нет данных',
                     'value': 'нет данных'}).to_csv(file_name, 
                                                         mode='a' if exists(file_name) else 'w', 
                                                         sep='\t', 
                                                         index=False,
                                                         header = not exists(file_name))
        i += 1
        with open('counter.txt', 'w') as f:
            f.write(str(i))
        time.sleep(1)
        continue
    
#   Аппендим данные в csv файл
    print(f'Спарсили {i} элемент')
    print('Пример: ',names[-1])
    print('Количество собранных элементов: ', len(names))
    print('Цена: ', values[-1])
    pd.DataFrame(data = {'lvl3_href': [url]*len(values),
                         'item_name': names,
                         'value': values}).to_csv(file_name, 
                                                         mode='a' if exists(file_name) else 'w', 
                                                         sep='\t', 
                                                         index=False,
                                                         header = not exists(file_name))
#     Обновляем файл с счетчиком спарсенных строк
    i += 1
    with open('counter_item_names.txt', 'w') as f:
        f.write(str(i))
    time.sleep(1)

else:
    os.remove('counter_item_names.txt')
    